In [1]:
#Importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

In [2]:
#Extracting academy_awards_1927-2015 as a Pandas DataFrame
academy_data_csv = "Resources/academy_awards_1927-2015.csv"
academy_data_df = pd.read_csv(academy_data_csv)
academy_data_df.head()

,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1927/1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1927/1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1927/1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


In [3]:
#Extracting academy_awards_1927-2015 as a Pandas DataFrame
imdb_top_csv = "Resources/imdb_top_1000.csv"
imdb_top_df = pd.read_csv(imdb_top_csv)
imdb_top_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


<h1>Transfrom Data</h1>

In [4]:
# Combine star columns into 1 combined column of actors
imdb_top_df['actors'] = imdb_top_df[imdb_top_df.columns[10:15]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

In [5]:
# Drop unnecessary columns
dopped_imdb_top_df = imdb_top_df.drop(columns=['Overview', 'Certificate', "Poster_Link", "Star1", "Star2", "Star3", "Star4"])
dopped_imdb_top_df.head()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Meta_score,Director,No_of_Votes,Gross,actors
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80.0,Frank Darabont,2343110,"28,341,469","Tim Robbins,Morgan Freeman,Bob Gunton,William ..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,100.0,Francis Ford Coppola,1620367,"134,966,411","Marlon Brando,Al Pacino,James Caan,Diane Keato..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84.0,Christopher Nolan,2303232,"534,858,444","Christian Bale,Heath Ledger,Aaron Eckhart,Mich..."
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,90.0,Francis Ford Coppola,1129952,"57,300,000","Al Pacino,Robert De Niro,Robert Duvall,Diane K..."
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,96.0,Sidney Lumet,689845,"4,360,000","Henry Fonda,Lee J. Cobb,Martin Balsam,John Fie..."


In [6]:
imdb_top_1000_df = dopped_imdb_top_df.rename(columns={"Series_Title": "film_name",
                                                "Released_Year": "released_year",
                                                "Runtime": "runtime",
                                                "Genre": "genre",
                                                "IMDB_Rating": "imdb_rating",
                                                "Meta_score": "meta_score",
                                                "Director": "director",
                                                "actors": "actors",
                                                "No_of_Votes": "votes",
                                                "Gross": "gross"
                                                })


In [7]:
imdb_top_1000_df

,film_name,released_year,runtime,genre,imdb_rating,meta_score,director,votes,gross,actors
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80.0,Frank Darabont,2343110,"28,341,469","Tim Robbins,Morgan Freeman,Bob Gunton,William ..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,100.0,Francis Ford Coppola,1620367,"134,966,411","Marlon Brando,Al Pacino,James Caan,Diane Keato..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84.0,Christopher Nolan,2303232,"534,858,444","Christian Bale,Heath Ledger,Aaron Eckhart,Mich..."
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,90.0,Francis Ford Coppola,1129952,"57,300,000","Al Pacino,Robert De Niro,Robert Duvall,Diane K..."
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,96.0,Sidney Lumet,689845,"4,360,000","Henry Fonda,Lee J. Cobb,Martin Balsam,John Fie..."
...,...,...,...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,1961,115 min,"Comedy, Drama, Romance",7.6,76.0,Blake Edwards,166544,NaN,"Audrey Hepburn,George Peppard,Patricia Neal,Bu..."
996,Giant,1956,201 min,"Drama, Western",7.6,84.0,George Stevens,34075,NaN,"Elizabeth Taylor,Rock Hudson,James Dean,Carrol..."
997,From Here to Eternity,1953,118 min,"Drama, Romance, War",7.6,85.0,Fred Zinnemann,43374,"30,500,000","Burt Lancaster,Montgomery Clift,Deborah Kerr,D..."
998,Lifeboat,1944,97 min,"Drama, War",7.6,78.0,Alfred Hitchcock,26471,NaN,"Tallulah Bankhead,John Hodiak,Walter Slezak,Wi..."
